In [3]:
from math import ceil
import tweepy as tw
import pandas as pd
from threading import Thread
from tqdm.auto import tqdm
import os
import json
from datetime import datetime

/home/haskari/.conda/envs/Twitter/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
def getTokens():
    token_arr = []
    with open('tokens') as f:
        tokens = f.read().strip().split('\n')
        for token in tokens:
            consumer_key,consumer_secret,access_token,access_token_secret = token.split('|')
            token_arr.append(dict(consumer_key=consumer_key,consumer_secret=consumer_secret,access_token=access_token,access_token_secret=access_token_secret))
    
    return (token_arr)

In [42]:
#testing functions
temp=getTokens()
token_dict=temp[3]
usersdf=pd.read_csv("Userlist.csv")
users=usersdf['0'].to_list()
for i in range(len(users)):
    users[i]=users[i].split('.')[0]
users=users[0:10]
#res1=returntweets(token_dict,users,count=100)
#res2=returnlikes(token_dict,users,count=100)
#res3=followeelist(token_dict,users)


 20%|██        | 2/10 [00:23<01:35, 11.91s/it]


KeyboardInterrupt: 

In [44]:
print(res3[1])

IndexError: list index out of range

In [16]:
def returntweets(token_dict, userid, count):

    def get_tweet_responses(consumer_key, consumer_secret, access_token, access_token_secret, user, num_tweets):

        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tw.API(auth, wait_on_rate_limit=True)

        #add sinceid here
        #open database, get sinceid as the max of the previous collected TwitterIDs and add as a parameter
        
       
        tweetsReq = tw.Cursor(api.user_timeline,
            user_id=user, count=num_tweets,
            exclude_replies=False, include_rts=True,
            tweet_mode='extended').items(num_tweets)
        tweets = []
        try:
            for tweet in tweetsReq:
                try:
                    full_text=tweet.retweeted_status.full_text
                except AttributeError:  # Not a Retweet
                    full_text=tweet.full_text
                tweets.append(dict(
                        full_text = full_text,
                        tweet_id=tweet.id_str,
                        screen_name=tweet.user.screen_name,
                        user_ID=user
                    ))
        except Exception as e:
            print(e)

        return tweets

    def getTweets(token_dict, userid, count):
        metalist=[]
        for user in tqdm(userid):

            

            tweets = get_tweet_responses(
                token_dict['consumer_key'],
                token_dict['consumer_secret'],
                token_dict['access_token'],
                token_dict['access_token_secret'],
                user, num_tweets=count
            )

            if len(tweets)==0:
                continue
            else:
                df = pd.DataFrame(tweets)
                metalist.append(df)
        return metalist

    return getTweets(token_dict,userid, count)


In [23]:
def returnlikes(token_dict, userid, count):
    def get_tweet_responses(consumer_key, consumer_secret, access_token, access_token_secret, user, num_tweets):
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tw.API(auth, wait_on_rate_limit=True)
        tweetsReq = tw.Cursor(api.get_favorites,
                user_id=user, count=num_tweets,
                tweet_mode='extended').items(num_tweets)
        tweets = []
        try:
            for tweet in tweetsReq:
                tweets.append(dict(
                        full_text = tweet.full_text,
                        tweet_id=tweet.id_str,
                        screen_name=tweet.user.screen_name,
                        Tweet_user_ID=tweet.user.id_str,
                        original_user_ID=user
                    ))
        except Exception as e:
            print(e)


        return tweets

    def getLikes(token_dict, listofusers,count):
        metalist=[]
        for user in tqdm(listofusers):
            tweets = get_tweet_responses(
                token_dict['consumer_key'],
                token_dict['consumer_secret'],
                token_dict['access_token'],
                token_dict['access_token_secret'],
                user, num_tweets=count
            )
            if len(tweets)==0:
                continue
            else:
                df = pd.DataFrame(tweets)
                metalist.append(df)
        return metalist
    return getLikes(token_dict,userid, count)

In [38]:
def followeelist(token_dict,listofusers):
    def get_tweet_responses(consumer_key, consumer_secret, access_token, access_token_secret, user):
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tw.API(auth, wait_on_rate_limit=True)
        tweetsReq = []
        try:
            for page in tw.Cursor(api.get_friend_ids,user_id=user,count=200).pages():
                tweetsReq.extend(page)


            tweets = []
        


            tweets.append(dict(
                Followers = tweetsReq,
                user_ID=user
            ))
        except Exception as e:
            print(e)


        return tweets

    def getFollowers(token_dict, listofusers):
        metalist=[]
        for user in tqdm(listofusers):
            tweets = get_tweet_responses(
                token_dict['consumer_key'],
                token_dict['consumer_secret'],
                token_dict['access_token'],
                token_dict['access_token_secret'],
                user
            )
            if len(tweets)==0:
                continue
            else:
                df = pd.DataFrame(tweets)
                metalist.append(df)
        return metalist
    return getFollowers(token_dict,listofusers)

In [ ]:
def replytobot(token_dict,tweetid,botusername):
    now = datetime.now()
    DATE = now.strftime("%Y-%m-%d")

    def get_tweet_responses(consumer_key, consumer_secret, access_token, access_token_secret, tweetid, num_tweets, date, botusername):
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tw.API(auth, wait_on_rate_limit=True)

        name = botusername
        tweet_id=tweetid

        tweetsReq = tw.Cursor(api.search_tweets,
                q='to: '+name + " -filter:retweets",
                lang="en", until=date,
                result_type='recent', tweet_mode='extended').items(num_tweets)

        tweets = []
        for tweet in tweetsReq:
            if hasattr(tweet, 'in_reply_to_status_id_str') == False:
                continue
            if (tweet.in_reply_to_status_id_str==tweet_id):
                tweets.append(dict(
                    full_text = tweet.full_text,
                    BotTweetID=tweet_id,
                    tweet_id=tweet.id_str,
                    screen_name=tweet.user.screen_name,
                    user_id=tweet.user.id_str
                    ))

        return tweets

    def getTweets(token_dict, tweetid, botusername):
        for tweet in tqdm(tweetid):
            tweets = get_tweet_responses(
                token_dict['consumer_key'],
                token_dict['consumer_secret'],
                token_dict['access_token'],
                token_dict['access_token_secret'],
                tweetid=tweet, num_tweets=20000, date=DATE, botusername=botusername
            )
            if len(tweets)==0:
                continue
            else:
                df = pd.DataFrame(tweets)
                return df
    return getTweets(token_dict, tweetid, botusername)

In [ ]:
def likeofreply():

In [ ]:
def linkclick():